# Notebook 1 : Évaluation Baseline des Modèles Pré-entraînés

**Cours:** M2 Datascale - Fouille de Données  

## Objectifs
- Évaluer 3 modèles pré-entraînés **SANS fine-tuning** sur SQuAD v1.1
- Établir une baseline de performance pour comparer avec les modèles fine-tunés
- Calculer les métriques F1 Score et Exact Match
- Mesurer le temps d'inférence

## Modèles Évalués
1. **DistilBERT-base-uncased** (66M paramètres)
2. **RoBERTa-base** (125M paramètres)
3. **DeBERTa-v3-base** (184M paramètres)

## Hypothèse
Les modèles pré-entraînés sans fine-tuning devraient obtenir des F1 scores autour de 40-45%, démontrant l'importance du fine-tuning qui devrait améliorer les performances de ~40-45 points.

## 1. Vérification de l'Environnement GPU

In [1]:
import torch

print("Vérification de l'environnement GPU...")
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU détecté: {device_name}")
    print(f"Mémoire disponible: {device_memory:.2f} GB")
    device = "cuda"
else:
    print("Aucun GPU détecté. Utilisation du CPU.")
    device = "cpu"

Vérification de l'environnement GPU...
GPU détecté: NVIDIA GeForce RTX 5080
Mémoire disponible: 16.60 GB


## 2. Installation des Dépendances

In [2]:
!pip install -q transformers datasets evaluate accelerate torch sentencepiece

## 3. Imports

In [3]:
import os
import json
import time
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DebertaV2Tokenizer,
    pipeline
)
import evaluate
from collections import defaultdict
from tqdm.auto import tqdm

/home/khaledbouabdallah/Projects/qa-finetuning-squad-webapp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4. Configuration

Nous allons évaluer chaque modèle sur le validation set de SQuAD v1.1

In [4]:
# Modèles à évaluer (pré-entraînés, SANS fine-tuning)
MODELS = {
    "distilbert": "distilbert-base-uncased",
    "roberta": "roberta-base",
    "deberta": "microsoft/deberta-v3-base"
}

# Configuration
MAX_LENGTH = 384
DOC_STRIDE = 128
BATCH_SIZE = 64

# Taille du dataset de validation
USE_FULL_DATASET = True
MAX_EVAL_SAMPLES = 10570 if USE_FULL_DATASET else 1000

print("="*80)
print("Configuration de l'évaluation baseline")
print("="*80)
print(f"Modèles: {list(MODELS.keys())}")
print(f"Échantillons de validation: {MAX_EVAL_SAMPLES}")
print(f"Device: {device}")
print("="*80)

Configuration de l'évaluation baseline
Modèles: ['distilbert', 'roberta', 'deberta']
Échantillons de validation: 10570
Device: cuda


## 5. Chargement du Dataset SQuAD

In [5]:
print("Chargement du dataset SQuAD v1.1...")
squad = load_dataset("squad", split="validation")

if not USE_FULL_DATASET:
    squad = squad.select(range(MAX_EVAL_SAMPLES))

print(f"Nombre d'exemples: {len(squad)}")
print(f"Colonnes: {squad.column_names}")
print(f"\nExemple:")
print(squad[0])

Chargement du dataset SQuAD v1.1...
Nombre d'exemples: 10570
Colonnes: ['id', 'title', 'context', 'question', 'answers']

Exemple:
{'id': '56be4db0acb8001400a502ec', 'title': 'Super_Bowl_50', 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'question': 'Whic

## 6. Fonction d'Évaluation

Cette fonction évalue un modèle sans fine-tuning sur le dataset SQuAD

In [ ]:
def evaluate_baseline_model(model_name, model_key):
    """
    Évalue un modèle baseline (pré-entraîné sans fine-tuning)

    Args:
        model_name: Nom du modèle HuggingFace
        model_key: Clé courte pour sauvegarder les résultats

    Returns:
        dict: Résultats (F1, EM, inference_time)
    """
    print("\n" + "="*80)
    print(f"Évaluation de {model_name} (BASELINE - pas de fine-tuning)")
    print("="*80)

    # Charger explicitement le tokenizer et le modèle
    print("Chargement du modèle et tokenizer...")
    if "deberta" in model_name.lower():
        tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    # Déplacer le modèle sur le GPU
    if device == "cuda":
        model = model.to(device)

    # Créer le pipeline QA avec le tokenizer et modèle chargés
    qa_pipeline = pipeline(
        "question-answering",
        model=model,
        tokenizer=tokenizer,
        device=0 if device == "cuda" else -1
    )

    # Préparer les prédictions
    predictions = []
    references = []
    inference_times = []

    print(f"\nInférence sur {len(squad)} exemples...")

    for idx, example in enumerate(tqdm(squad)):
        # Extraire les données
        context = example['context']
        question = example['question']
        answer = example['answers']['text'][0]
        answer_start = example['answers']['answer_start'][0]

        # Faire la prédiction avec mesure du temps
        start_time = time.time()
        try:
            result = qa_pipeline(question=question, context=context)
            inference_time = time.time() - start_time
            inference_times.append(inference_time)

            # Stocker les résultats
            predictions.append({
                'id': example['id'],
                'prediction_text': result['answer']
            })
        except Exception as e:
            # En cas d'erreur, utiliser une réponse vide
            inference_time = time.time() - start_time
            inference_times.append(inference_time)
            predictions.append({
                'id': example['id'],
                'prediction_text': ''
            })

        references.append({
            'id': example['id'],
            'answers': example['answers']
        })

        # Afficher la progression
        if (idx + 1) % 500 == 0:
            avg_time = np.mean(inference_times[-500:])
            print(f"  Traité {idx + 1}/{len(squad)} exemples (avg time: {avg_time:.3f}s)")

    # Calculer les métriques
    print("\nCalcul des métriques...")
    squad_metric = evaluate.load("squad")

    # Formater pour la métrique
    formatted_predictions = [
        {'id': p['id'], 'prediction_text': p['prediction_text']}
        for p in predictions
    ]
    formatted_references = [
        {'id': r['id'], 'answers': r['answers']}
        for r in references
    ]

    metrics = squad_metric.compute(
        predictions=formatted_predictions,
        references=formatted_references
    )

    # Ajouter le temps d'inférence
    metrics['avg_inference_time'] = float(np.mean(inference_times))
    metrics['total_inference_time'] = float(np.sum(inference_times))

    # Afficher les résultats
    print("\n" + "-"*80)
    print(f"RÉSULTATS - {model_name} (BASELINE)")
    print("-"*80)
    print(f"F1 Score: {metrics['f1']:.2f}%")
    print(f"Exact Match: {metrics['exact_match']:.2f}%")
    print(f"Temps moyen par exemple: {metrics['avg_inference_time']:.3f}s")
    print(f"Temps total: {metrics['total_inference_time']:.2f}s")
    print("-"*80)

    # Sauvegarder les résultats
    output_dir = f"../models/{model_key}_baseline"
    os.makedirs(output_dir, exist_ok=True)

    results_file = os.path.join(output_dir, "results.json")
    results_to_save = {
        "model_name": model_name,
        "model_type": model_key,
        "finetuned": False,
        "f1": metrics['f1'],
        "exact_match": metrics['exact_match'],
        "avg_inference_time": metrics['avg_inference_time'],
        "total_inference_time": metrics['total_inference_time'],
        "num_eval_samples": len(squad)
    }

    with open(results_file, 'w') as f:
        json.dump(results_to_save, f, indent=2)

    print(f"\n✓ Résultats sauvegardés dans: {results_file}")

    return metrics

In [9]:
MODELS

{'distilbert': 'distilbert-base-uncased',
 'roberta': 'roberta-base',
 'deberta': 'microsoft/deberta-v3-base'}

## 7. Évaluation de Tous les Modèles Baseline

In [ ]:
# Stocker tous les résultats
all_results = {}

for model_key, model_name in MODELS.items():
    try:
        metrics = evaluate_baseline_model(model_name, model_key)
        all_results[model_key] = metrics
    except Exception as e:
        print(f"\n❌ Erreur lors de l'évaluation de {model_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("ÉVALUATION BASELINE TERMINÉE")
print("="*80)


Évaluation de microsoft/deberta-v3-base (BASELINE - pas de fine-tuning)
Chargement du modèle et tokenizer...


Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0



Inférence sur 10570 exemples...


  5%|▍         | 500/10570 [00:24<07:25, 22.62it/s]

  Traité 500/10570 exemples (avg time: 0.049s)


...

  Traité 10000/10570 exemples (avg time: 0.046s)


 99%|█████████▉| 10498/10570 [08:06<00:03, 21.65it/s]

  Traité 10500/10570 exemples (avg time: 0.047s)


100%|██████████| 10570/10570 [08:09<00:00, 21.60it/s]



Calcul des métriques...

--------------------------------------------------------------------------------
RÉSULTATS - microsoft/deberta-v3-base (BASELINE)
--------------------------------------------------------------------------------
F1 Score: 5.95%
Exact Match: 0.20%
Temps moyen par exemple: 0.046s
Temps total: 485.39s
--------------------------------------------------------------------------------

✓ Résultats sauvegardés dans: ../models/deberta_baseline/results.json

ÉVALUATION BASELINE TERMINÉE


## 8. Tableau Récapitulatif des Performances Baseline

In [10]:
import pandas as pd

# Charger les résultats depuis les fichiers JSON sauvegardés
summary_data = []

for model_key in MODELS.keys():
    results_file = f"../models/{model_key}_baseline/results.json"
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            metrics = json.load(f)
        summary_data.append({
            'Modèle': model_key.upper(),
            'F1 (%)': f"{metrics['f1']:.2f}",
            'EM (%)': f"{metrics['exact_match']:.2f}",
            'Temps/exemple (s)': f"{metrics['avg_inference_time']:.3f}"
        })
    else:
        print(f"⚠️  Fichier non trouvé: {results_file}")

if summary_data:
    df_summary = pd.DataFrame(summary_data)
    print("\n" + "="*80)
    print("RÉSUMÉ DES PERFORMANCES BASELINE (pré-entraînés sans fine-tuning)")
    print("="*80)
    print(df_summary.to_string(index=False))
    print("="*80)

    print("\n💡 Ces scores baseline serviront de référence pour mesurer")
    print("   l'amélioration apportée par le fine-tuning sur SQuAD.")
    print("\n📊 Amélioration attendue après fine-tuning: +40-45 points de F1")
else:
    print("❌ Aucun résultat disponible")


RÉSUMÉ DES PERFORMANCES BASELINE (pré-entraînés sans fine-tuning)
    Modèle F1 (%) EM (%) Temps/exemple (s)
DISTILBERT   6.82   0.40             0.004
   ROBERTA   6.50   1.25             0.006
   DEBERTA   5.95   0.20             0.046

💡 Ces scores baseline serviront de référence pour mesurer
   l'amélioration apportée par le fine-tuning sur SQuAD.

📊 Amélioration attendue après fine-tuning: +40-45 points de F1



Les résultats obtenus mettent en évidence des différences notables entre les modèles évalués, tant en termes de performance que de coût computationnel.

Le modèle DistilBERT, bien que conçu pour être léger et rapide, présente les scores Exact Match et F1 les plus faibles. Ce comportement est attendu, car la distillation réduit la capacité du modèle à capturer des dépendances contextuelles complexes. En revanche, son temps d’inférence moyen est le plus faible, ce qui en fait un candidat pertinent pour des applications nécessitant des contraintes fortes de latence.

RoBERTa-base obtient des performances intermédiaires, avec une amélioration nette des scores par rapport à DistilBERT. Cette progression s’explique par une capacité de représentation plus élevée et un pré-entraînement plus riche. Toutefois, cette amélioration se fait au prix d’un temps d’inférence plus important, illustrant un compromis classique entre performance et complexité du modèle.

DeBERTa-v3-base se distingue par les meilleurs résultats en Exact Match et en F1-score. Sa supériorité peut être attribuée à son architecture améliorée, notamment l’utilisation de mécanismes de désentrelacement des représentations de contenu et de position, favorisant une compréhension plus fine du contexte. Néanmoins, ce gain de performance s’accompagne d’un coût computationnel plus élevé, comme en témoigne son temps d’inférence moyen.

Globalement, ces résultats confirment l’existence d’un compromis clair entre précision et efficacité computationnelle. Ils justifient la poursuite du travail avec une phase de fine-tuning supervisé, dont l’objectif sera d’améliorer les performances tout en analysant l’impact de l’entraînement sur ce compromis.

## 9. Conclusion

Les modèles baseline (pré-entraînés sans fine-tuning) ont été évalués. Ces résultats serviront de point de comparaison pour démontrer l'efficacité du fine-tuning.

**Prochaines étapes:**
1. Fine-tuner chaque modèle sur SQuAD (notebooks 02, 03, 04)
2. Comparer les performances baseline vs fine-tuned (notebook 05)
3. Analyser le gain de performance apporté par le fine-tuning